# 1. Assume that we have two convolution kernels of size $k_1$ and $k_2$, respectively (with no nonlinearity in between).

## 1.1 Prove that the result of the operation can be expressed by a single convolution.

In [40]:
K_2 = torch.normal(0, 1, (3, 3), requires_grad=True)
train_Y = corr2d(corr2d(X,K),K)
y_pred = corr2d(X,K_2)
lr = 1e-3  # Learning rate

for i in range(10):
    Y_hat = corr2d(X,K_2)
    l = (Y_hat - train_Y) ** 2
    # print(l)
    
    l.sum().backward()
    # print(K_2.grad)
    # Update the kernel
    K_2.data -= lr * K_2.grad
    K_2.grad.zero_()
    if (i + 1) % 2 == 0:
        print(f'epoch {i + 1}, loss {l.sum():.3f}')
    # print(1,K_2)
K_2

epoch 2, loss 9.327
epoch 4, loss 0.143
epoch 6, loss 0.002
epoch 8, loss 0.000
epoch 10, loss 0.000


tensor([[ 1.7815,  0.2595,  0.7211],
        [ 0.4847, -1.4443, -0.1710],
        [ 0.4958,  1.8198,  0.0529]], requires_grad=True)

In [96]:
import torch
import torch.nn.functional as F
def corr2d(X,K):
    h,w = K.shape
    Y = torch.zeros(X.shape[0]-h+1, X.shape[1]-w+1)
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            Y[i, j] = (X[i:i+h, j:j+w]*K).sum()
    return Y

X = torch.randn(size=(8, 8))
# K_1 = torch.tensor([[1,0],[0,1]])
K_1 = torch.randn(2, 2)
K_1_1 = torch.randn(3, 3)
# print(K_1)
# print(corr2d(corr2d(X,K_1),K_1))
# K_2 = torch.tensor([[4/3,0,0],[0,4/3,0],[0,0,4/3]])
K_2 = F.conv_transpose2d(K_1.reshape(1,1,K_1.shape[0],-1), K_1_1.reshape(1,1,K_1_1.shape[0],-1)).squeeze()
print(K_1.shape,K_1_1.shape,K_2.shape)
print((corr2d(X,K_2)-corr2d(corr2d(X,K_1),K_1_1)< 1e-6).all())

torch.Size([2, 2]) torch.Size([3, 3]) torch.Size([4, 4])
tensor(True)


## 1.2 What is the dimensionality of the equivalent single convolution?

$(k_1.h+k_2.h-1,k_1.w+k_2.w-1)$

## 1.3 Is the converse true, i.e., can you always decompose a convolution into two smaller ones?

In [61]:
K_2 = torch.randn(3,3)
K_1 = torch.randn(2,2)
K_1_1 = corr2d(K_2,K_1)
corr2d(corr2d(X,K_1_1),K_1)

tensor([[-20.7627, -20.7627, -20.7627, -20.7627, -20.7627, -20.7627],
        [-20.7627, -20.7627, -20.7627, -20.7627, -20.7627, -20.7627],
        [-20.7627, -20.7627, -20.7627, -20.7627, -20.7627, -20.7627],
        [-20.7627, -20.7627, -20.7627, -20.7627, -20.7627, -20.7627],
        [-20.7627, -20.7627, -20.7627, -20.7627, -20.7627, -20.7627],
        [-20.7627, -20.7627, -20.7627, -20.7627, -20.7627, -20.7627]])

In [62]:
corr2d(corr2d(X,K_1),K_1_1)

tensor([[-20.7627, -20.7627, -20.7627, -20.7627, -20.7627, -20.7627],
        [-20.7627, -20.7627, -20.7627, -20.7627, -20.7627, -20.7627],
        [-20.7627, -20.7627, -20.7627, -20.7627, -20.7627, -20.7627],
        [-20.7627, -20.7627, -20.7627, -20.7627, -20.7627, -20.7627],
        [-20.7627, -20.7627, -20.7627, -20.7627, -20.7627, -20.7627],
        [-20.7627, -20.7627, -20.7627, -20.7627, -20.7627, -20.7627]])

In [63]:
corr2d(X,K_2)

tensor([[-4.5812, -4.5812, -4.5812, -4.5812, -4.5812, -4.5812],
        [-4.5812, -4.5812, -4.5812, -4.5812, -4.5812, -4.5812],
        [-4.5812, -4.5812, -4.5812, -4.5812, -4.5812, -4.5812],
        [-4.5812, -4.5812, -4.5812, -4.5812, -4.5812, -4.5812],
        [-4.5812, -4.5812, -4.5812, -4.5812, -4.5812, -4.5812],
        [-4.5812, -4.5812, -4.5812, -4.5812, -4.5812, -4.5812]])

# 2. Assume an input of shape 
 and a convolution kernel of shape 
, padding of 
, and stride of 
.

What is the computational cost (multiplications and additions) for the forward propagation?

What is the memory footprint?

What is the memory footprint for the backward computation?

What is the computational cost for the backpropagation?



# 3. By what factor does the number of calculations increase if we double both the number of input channels 
 and the number of output channels 
? What happens if we double the padding?



# 4. Are the variables Y1 and Y2 in the final example of this section exactly the same? Why?



# 5. Express convolutions as a matrix multiplication, even when the convolution window is not 
.



# 6. Your task is to implement fast convolutions with a 
 kernel. One of the algorithm candidates is to scan horizontally across the source, reading a 
-wide strip and computing the 
-wide output strip one value at a time. The alternative is to read a 
 wide strip and compute a 
-wide output strip. Why is the latter preferable? Is there a limit to how large you should choose 
?



# 7. Assume that we have a 
 matrix.

How much faster is it to multiply with a block-diagonal matrix if the matrix is broken up into 
 blocks?

What is the downside of having 
 blocks? How could you fix it, at least partly?